#### Setup

In [ ]:
!pip install -U tensorflow keras

In [ ]:
!pip install -U talos

In [ ]:
# Import general Python libraries
import pandas as pd
import numpy as np
import random
import sklearn
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [ ]:
# Specify seeds for random-operations
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
# Import sklearn-specific modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [ ]:
# Import tensorflow-specific modules
import tensorflow as tf
tf.random.set_seed(seed_value)
print("Tensorflow Version: {}".format(tf.__version__))
print("Keras Version: {}".format(tf.keras.__version__))

Tensorflow Version: 2.4.1
Keras Version: 2.4.0


In [ ]:
# Import keras-specific modules
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adadelta, RMSprop, Adam, Adamax, Nadam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform, HeNormal, HeUniform, LecunNormal, LecunUniform
from tensorflow.keras.metrics import AUC, Precision, Recall

In [ ]:
# Import talos-specific modules
import talos

In [ ]:
# Set pandas options
pd.set_option("display.max_columns", None)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### a) Prepare Dataset

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Structured.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 35)
246891


,campaign_successful,goal,number_of_collaborators,funding_period,days_between_created_and_launched,launch_quartal,staff_pick,campaign_has_demo_video,campaign_has_environmental_commitments,number_of_images,number_of_videos,number_of_audios,number_of_interactives,number_of_words,number_of_links,creator_verified_identity,creator_fb_auth,creator_has_image,creator_allows_follows,number_of_creator_backings,number_of_creator_projects,facebook_linked,twitter_linked,instagram_linked,linkedin_linked,number_of_rewards,number_of_words_per_reward,lowest_pledge_level,highest_pledge_level,has_limited_rewards,has_shipped_rewards,has_restricted_shipping_rewards,avg_months_until_reward,location,category
22821161,0,50000.0,0,45,102,3,0,1,0,13,0,0,0,1378,2,1,0,1,1,0,1,0,0,0,0,8,36.75,5.0,1417.0,0,1,1,5.5,United States,Design_Product Design


In [ ]:
# Convert categorical features into dummy-variables
print("Before: {}".format(kickstarter_df.shape))
kickstarter_df["launch_quartal"] = kickstarter_df.launch_quartal.apply(str)
kickstarter_df = pd.get_dummies(kickstarter_df, prefix=["launch_quartal", "location", "category"], columns=["launch_quartal", "location", "category"], drop_first=False)
print("After: {}".format(kickstarter_df.shape))
kickstarter_df.head()

Before: (246891, 35)
After: (246891, 234)


,campaign_successful,goal,number_of_collaborators,funding_period,days_between_created_and_launched,staff_pick,campaign_has_demo_video,campaign_has_environmental_commitments,number_of_images,number_of_videos,number_of_audios,number_of_interactives,number_of_words,number_of_links,creator_verified_identity,creator_fb_auth,creator_has_image,creator_allows_follows,number_of_creator_backings,number_of_creator_projects,facebook_linked,twitter_linked,instagram_linked,linkedin_linked,number_of_rewards,number_of_words_per_reward,lowest_pledge_level,highest_pledge_level,has_limited_rewards,has_shipped_rewards,has_restricted_shipping_rewards,avg_months_until_reward,launch_quartal_1,launch_quartal_2,launch_quartal_3,launch_quartal_4,location_Africa,location_Australia,location_Belgium,location_Canada,location_China,location_Denmark,location_France,location_Germany,location_Hong Kong,location_Ireland,location_Italy,location_Japan,location_Latin and South America,location_Mexico,location_Netherlands,location_New Zealand,location_No Location,location_Norway,location_Oceania and Antarctica,location_Rest of Asia,location_Rest of Europe,location_Singapore,location_Spain,location_Sweden,location_Switzerland,location_United Kingdom,location_United States,category_Art_Ceramics,category_Art_Conceptual Art,category_Art_Digital Art,category_Art_Illustration,category_Art_Installations,category_Art_Mixed Media,category_Art_No Subcategory,category_Art_Painting,category_Art_Performance Art,category_Art_Public Art,category_Art_Sculpture,category_Art_Social Practice,category_Art_Textiles,category_Art_Video Art,category_Comics_Anthologies,category_Comics_Comic Books,category_Comics_Events,category_Comics_Graphic Novels,category_Comics_No Subcategory,category_Comics_Webcomics,category_Crafts_Candles,category_Crafts_Crochet,category_Crafts_DIY,category_Crafts_Embroidery,category_Crafts_Glass,category_Crafts_Knitting,category_Crafts_No Subcategory,category_Crafts_Pottery,category_Crafts_Printing,category_Crafts_Quilts,category_Crafts_Stationery,category_Crafts_Taxidermy,category_Crafts_Weaving,category_Crafts_Woodworking,category_Dance_No Subcategory,category_Dance_Performances,category_Dance_Residencies,category_Dance_Spaces,category_Dance_Workshops,category_Design_Architecture,category_Design_Civic Design,category_Design_Graphic Design,category_Design_Interactive Design,category_Design_No Subcategory,category_Design_Product Design,category_Design_Toys,category_Design_Typography,category_Fashion_Accessories,category_Fashion_Apparel,category_Fashion_Childrenswear,category_Fashion_Couture,category_Fashion_Footwear,category_Fashion_Jewelry,category_Fashion_No Subcategory,category_Fashion_Pet Fashion,category_Fashion_Ready-to-wear,category_Film & Video_Action,category_Film & Video_Animation,category_Film & Video_Comedy,category_Film & Video_Documentary,category_Film & Video_Drama,category_Film & Video_Experimental,category_Film & Video_Family,category_Film & Video_Fantasy,category_Film & Video_Festivals,category_Film & Video_Horror,category_Film & Video_Movie Theaters,category_Film & Video_Music Videos,category_Film & Video_Narrative Film,category_Film & Video_No Subcategory,category_Film & Video_Romance,category_Film & Video_Science Fiction,category_Film & Video_Shorts,category_Film & Video_Television,category_Film & Video_Thrillers,category_Film & Video_Webseries,category_Food_Bacon,category_Food_Community Gardens,category_Food_Cookbooks,category_Food_Drinks,category_Food_Events,category_Food_Farmer's Markets,category_Food_Farms,category_Food_Food Trucks,category_Food_No Subcategory,category_Food_Restaurants,category_Food_Small Batch,category_Food_Spaces,category_Food_Vegan,category_Games_Gaming Hardware,category_Games_Live Games,category_Games_Mobile Games,category_Games_No Subcategory,category_Games_Playing Cards,category_Games_Puzzles,category_Games_Tabletop Games,category_Games_Video Games,category_Journalism_Audio,category_Journalism_No Subcategory,category_

In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
kickstarter_notarget_df = kickstarter_df.drop(columns=["campaign_successful"])
X = kickstarter_notarget_df.astype("float32").to_numpy()

print(type(y))
print(y.shape)
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891, 233)


In [ ]:
# Retrieve column names which will be used for later pre-processing and evaluation
feature_names = kickstarter_notarget_df.columns.values
numeric_feature_names = ["goal", "number_of_collaborators", "funding_period", "days_between_created_and_launched", "number_of_images", "number_of_videos", "number_of_audios", "number_of_interactives", "number_of_words", "number_of_links", "number_of_creator_backings", "number_of_creator_projects", "number_of_rewards", "number_of_words_per_reward", "lowest_pledge_level", "highest_pledge_level", "avg_months_until_reward"]
binary_feature_names = [x for x in feature_names if x not in numeric_feature_names]
numeric_features = [kickstarter_notarget_df.columns.get_loc(x) for x in numeric_feature_names]
binary_features = [kickstarter_notarget_df.columns.get_loc(x) for x in binary_feature_names]

In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
X_subtrain, X_val, y_subtrain, y_val = train_test_split(X_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(X_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(X_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858, 233)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824, 233)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034, 233)
Shape of y_val: (37034,)
Shape of X_test: (37033, 233)
Shape of y_test: (37033,)


#### b) Baseline

This model provides the baseline, which will be used as starting point for the hyperparameter search. The goal is to gradually improve the model performance.

**Hyperparamaters:**
- Untransformed Dataset
- Regularizer = None
- Optimizer = RMSprop
- Weight Initialization = GlorotNormal
- Batch Size = 256
- Epochs = 20

In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 113.0285 - binary_accuracy: 0.6713 - val_loss: 2.1117 - val_binary_accuracy: 0.7044
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 2.1456 - binary_accuracy: 0.6860 - val_loss: 6.4972 - val_binary_accuracy: 0.3995
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 2.2455 - binary_accuracy: 0.6893 - val_loss: 0.8982 - val_binary_accuracy: 0.7402
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 2.2477 - binary_accuracy: 0.6916 - val_loss: 1.5085 - val_binary_accuracy: 0.7364
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 2.1612 - binary_accuracy: 0.6998 - val_loss: 0.7620 - val_binary_accuracy: 0.7607
Epoch 6/100
676/676 [==============================] - 1s 2ms/step - loss: 2.1460 - binary_accuracy: 0.7027 - val_loss: 3.9274 - val_binary_accuracy: 0.6998
Epoch 7/100
676/676 [==============================] - 1

#### c) Test Different Preprocessing Strategies

##### 1. Standardization: Numerical + Binary Attributes

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", StandardScaler(), numeric_features),
      ("binary", StandardScaler(), binary_features)
    ],
    remainder="drop", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-0.298547,-0.198447,-0.941790,-0.301747,-0.542916,-0.264041,-0.08432,-0.01557,-1.130397,-0.746823,-0.215401,-0.224846,-0.265058,-0.639590,-0.094083,-0.563342,0.613136,-0.287578,0.675031,-0.08672,-1.081998,-0.947352,0.147337,0.078146,-0.654237,-0.432702,-0.33426,-0.159431,-1.120286,-1.49029,-0.673015,1.766646,-0.594752,-0.607508,-0.541083,-0.033,-0.145087,-0.04121,-0.199623,-0.036584,-0.052331,-0.070387,-0.097759,-0.057777,-0.052387,-0.094291,-0.048765,-0.053432,-0.048346,-0.084526,-0.063451,-0.059955,-0.040139,-0.009622,-0.064318,-0.085869,-0.051829,-0.072474,-0.066504,-0.041839,-0.324079,0.561241,-0.033436,-0.054723,-0.069468,-0.101491,-0.038213,-0.092748,-0.155436,-0.097396,-0.077544,-0.09176,-0.072312,-0.011024,-0.031379,-0.025692,-0.036663,-0.096636,-0.021654,-0.075865,-0.109888,-0.044136,-0.038741,-0.022442,-0.059613,-0.019693,-0.020697,-0.022052,-0.111494,-0.017181,-0.027542,-0.012955,-0.026796,-0.004811,-0.015215,-0.060052,-0.08109,-0.055933,-0.014027,-0.025006,-0.021385,-0.045878,-0.030725,-0.072996,-0.034038,-0.092335,-0.223068,-0.012266,-0.015591,-0.104757,-0.150343,-0.038666,-0.028676,-0.053377,-0.064273,-0.144796,-0.020415,-0.054989,-0.045752,-0.082381,-0.07941,-0.196261,-0.079263,-0.039557,-0.032647,-0.03025,-0.028978,-0.061205,-0.028166,-0.044201,-0.113237,-0.162721,-0.024656,-0.043805,5.617208,-0.053649,-0.048645,-0.12157,-0.018944,-0.028978,-0.038891,-0.090241,-0.042459,-0.036663,-0.060439,-0.076173,-0.166301,-0.097909,-0.072916,-0.039336,-0.047128,-0.031655,-0.055568,-0.089324,-0.078221,-0.088331,-0.029078,-0.172891,-0.151786,-0.035539,-0.068622,-0.022442,-0.045624,-0.034038,-0.058578,-0.028371,-0.009001,-0.084317,-0.0125,-0.114203,-0.07728,-0.057727,-0.106438,-0.124334,-0.073833,-0.030058,-0.020273,-0.047128,-0.215988,-0.095994,-0.03082,-0.03941,-0.138353,-0.076938,-0.026579,-0.043406,-0.040067,-0.122254,-0.055881,-0.065887,-0.043473,-0.053812,-0.035293,-0.08456,-0.0322,-0.139306,-0.02152,-0.156196,-0.012955,-0.029572,-0.016317,-0.120561,-0.147604,-0.054936,-0.064364,-0.053541,-0.02152,-0.049416,-0.036105,-0.03963,-0.14633,-0.029767,-0.048105,-0.026579,-0.031928,-0.089619,-0.092685,-0.025006,-0.11784,-0.037832,-0.093192,-0.040284,-0.029177,-0.054403,-0.103594,-0.025579,-0.0322,-0.040067,-0.0322,-0.051037,-0.141534,-0.066985,-0.022183
1,-0.307414,-0.198447,-2.019196,-0.326723,-0.542916,-0.264041,-0.08432,-0.01557,-0.369989,0.009728,-0.028455,-0.224846,-0.636566,-0.636051,-0.067116,-0.563342,-0.513072,-0.287578,0.675031,-0.08672,-1.081998,1.055574,0.147337,0.078146,-0.654237,-0.432702,-0.33426,6.272299,-1.120286,0.67101,1.485850,1.766646,-0.594752,-0.607508,-0.541083,-0.033,-0.145087,-0.04121,-0.199623,-0.036584,-0.052331,-0.070387,-0.097759,-0.057777,-0.052387,-0.094291,-0.048765,-0.053432,-0.048346,-0.084526,-0.063451,-0.059955,-0.040139,-0.009622,-0.064318,-0.085869,-0.051829,-0.072474,-0.066504,-0.041839,-0.324079,0.561241,-0.033436,-0.054723,-0.069468,-0.101491,-0.038213,-0.092748,-0.155436,-0.097396,-0.077544,-0.09176,-0.072312,-0.011024,-0.031379,-0.025692,-0.036663,-0.096636,-0.021654,-0.075865,-0.109888,-0.044136,-0.038741,-0.022442,-0.059613,-0.019693,-0.020697,-0.022052,-0.111494,-0.017181,-

In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val) ,batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 0.6629 - binary_accuracy: 0.6449 - val_loss: 0.5172 - val_binary_accuracy: 0.7577
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5015 - binary_accuracy: 0.7620 - val_loss: 0.5072 - val_binary_accuracy: 0.7609
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4976 - binary_accuracy: 0.7636 - val_loss: 0.5024 - val_binary_accuracy: 0.7620
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4918 - binary_accuracy: 0.7653 - val_loss: 0.4989 - val_binary_accuracy: 0.7642
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4939 - binary_accuracy: 0.7654 - val_loss: 0.4956 - val_binary_accuracy: 0.7659
Epoch 6/100
676/676 [==============================] - 2s 2ms/step - loss: 0.4911 - binary_accuracy: 0.7669 - val_loss: 0.4939 - val_binary_accuracy: 0.7663
Epoch 7/100
676/676 [==============================] - 2s 

##### 2. Standardization: Only Numerical Attributes

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", StandardScaler(), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-0.298547,-0.198447,-0.941790,-0.301747,-0.542916,-0.264041,-0.08432,-0.01557,-1.130397,-0.746823,-0.215401,-0.224846,-0.265058,-0.639590,-0.094083,-0.563342,0.613136,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.307414,-0.198447,-2.019196,-0.326723,-0.542916,-0.264041,-0.08432,-0.01557,-0.369989,0.009728,-0.028455,-0.224846,-0.636566,-0.636051,-0.067116,-0.563342,-0.513072,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.288786,-0.198447,-0.326129,1.762865,0.762853,0.903184,-0.08432,-0.01557,3.482626,0.261912,1.168003,0.223308,-0.450812,0.610507,-0.094083,-0.587589,-0.400451,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val) ,batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 0.6023 - binary_accuracy: 0.6623 - val_loss: 0.5160 - val_binary_accuracy: 0.7394
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5066 - binary_accuracy: 0.7508 - val_loss: 0.4949 - val_binary_accuracy: 0.7577
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4918 - binary_accuracy: 0.7611 - val_loss: 0.4862 - val_binary_accuracy: 0.7621
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4842 - binary_accuracy: 0.7642 - val_loss: 0.4825 - val_binary_accuracy: 0.7634
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4826 - binary_accuracy: 0.7653 - val_loss: 0.4802 - val_binary_accuracy: 0.7658
Epoch 6/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4805 - binary_accuracy: 0.7667 - val_loss: 0.4789 - val_binary_accuracy: 0.7678
Epoch 7/100
676/676 [==============================] - 1s 

Result: Standardizing Attributes (both Numerical + Binary or Only Numerical) helped to increase performance of the model

##### 3. Normalization: Numerical + Binary Features

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", Pipeline([
                            ("normalization", StandardScaler()),
                            ("standarization", MinMaxScaler(feature_range=(-1,1)))
                          ]), numeric_features),
      ("binary", MinMaxScaler(feature_range=(-1,1)), binary_features)
    ],
    remainder="drop", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-0.999650,-1.0,-0.567010,-0.995506,-1.00000,-1.000000,-1.0,-1.0,-0.988894,-1.000000,-1.000000,-1.000000,-0.947137,-0.929240,-0.999815,-0.99200,-0.777778,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-0.999862,-1.0,-0.855670,-0.997432,-1.00000,-1.000000,-1.0,-1.0,-0.918719,-0.960265,-0.994299,-1.000000,-0.964758,-0.928947,-0.998155,-0.99200,-0.916667,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-0.999416,-1.0,-0.402062,-0.836276,-0.87027,-0.935484,-1.0,-1.0,-0.563174,-0.947020,-0.957811,-0.963636,-0.955947,-0.825965,-0.999815,-0.99712,-0.902778,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-

In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val) ,batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 0.6423 - binary_accuracy: 0.6496 - val_loss: 0.5990 - val_binary_accuracy: 0.6765
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5801 - binary_accuracy: 0.6990 - val_loss: 0.5730 - val_binary_accuracy: 0.7116
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5658 - binary_accuracy: 0.7084 - val_loss: 0.5886 - val_binary_accuracy: 0.6878
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5575 - binary_accuracy: 0.7141 - val_loss: 0.5552 - val_binary_accuracy: 0.7143
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5520 - binary_accuracy: 0.7206 - val_loss: 0.5480 - val_binary_accuracy: 0.7241
Epoch 6/100
676/676 [==============================] - 1s 2ms/step - loss: 0.5481 - binary_accuracy: 0.7225 - val_loss: 0.5471 - val_binary_accuracy: 0.7271
Epoch 7/100
676/676 [==============================] - 1s 

Result: Normalization is not helpful

##### 4. Power Transformation of Numerical Features

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", PowerTransformer(method="yeo-johnson", standardize=True), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-1.083699,-0.239021,-0.986135,-0.190185,-0.933752,-0.366213,-0.13909,-0.025692,-2.667854,-1.554894,-0.878420,-0.612750,-0.063636,-0.619896,-1.382878,-0.74978,0.866256,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.621048,-0.239021,-2.927746,-0.514813,-0.933752,-0.366213,-0.13909,-0.025692,-0.148796,0.467339,1.228547,-0.612750,-0.601676,-0.613763,0.471788,-0.74978,-0.472150,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.768322,-0.239021,-0.229063,1.802791,1.300685,2.727485,-0.13909,-0.025692,2.257514,0.751221,1.769156,1.758142,-0.313814,0.831888,-1.382878,-1.25173,-0.290671,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val) ,batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 0.5602 - binary_accuracy: 0.7040 - val_loss: 0.4566 - val_binary_accuracy: 0.7780
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4468 - binary_accuracy: 0.7823 - val_loss: 0.4371 - val_binary_accuracy: 0.7894
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4322 - binary_accuracy: 0.7905 - val_loss: 0.4316 - val_binary_accuracy: 0.7916
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4284 - binary_accuracy: 0.7924 - val_loss: 0.4297 - val_binary_accuracy: 0.7918
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4265 - binary_accuracy: 0.7934 - val_loss: 0.4284 - val_binary_accuracy: 0.7935
Epoch 6/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4244 - binary_accuracy: 0.7954 - val_loss: 0.4279 - val_binary_accuracy: 0.7934
Epoch 7/100
676/676 [==============================] - 1s 

Result: Transforming Numerical Features into a Normal Distribution helps extremely

##### 5. Decorrelation with PCA

In [ ]:
# Specify pre-processing pipeline for the different types of attributes
preprocessing = ColumnTransformer(
    [
      ("numeric", Pipeline([
                            ("powertransform", PowerTransformer(method="yeo-johnson", standardize=True)),
                            ("pca", PCA())
                          ]), numeric_features),
     ("binary", PCA(), binary_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain_transformed = preprocessing.transform(X_subtrain)
X_val_transformed = preprocessing.transform(X_val)

In [ ]:
# Validation
tmp = pd.DataFrame(X_subtrain_transformed)
print(tmp.shape)
tmp.head(3)

(172824, 233)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232
0,-2.601059,-0.410007,-0.404783,1.164287,-0.091849,-0.212521,-0.529216,2.294785,0.580161,0.953602,-0.148654,-0.430559,0.612913,0.851988,-0.351759,0.900483,0.139061,-1.099618,-0.327502,-0.019383,-0.226823,0.747183,-0.429928,-0.461063,0.082005,-0.414749,0.110693,-0.026495,0.099997,0.105971,-0.116056,-0.053963,0.024474,-0.122303,-0.250888,0.853770,0.142416,-0.004165,0.074606,0.108945,-0.018510,0.015295,0.043422,0.028247,0.006209,0.025120,-0.063271,-0.086629,0.007027,-0.002120,-0.012182,0.042487,-0.003254,0.006190,0.001184,-0.001217,-0.012217,0.009180,0.002767,0.001048,-0.004724,-0.011325,-0.002727,-0.006939,0.000472,0.006187,0.000041,0.000298,0.004510,0.010738,0.006230,0.001477,0.003657,0.002938,-0.000782,-0.001346,-0.000818,0.003122,-0.005166,0.003877,0.005918,0.005787,-0.005882,0.00263,-0.002904,-0.005847,-0.003477,0.000527,0.000739,0.004867,0.000880,-0.005517,0.002925,-0.000497,0.003080,0.007818,-0.001841,0.002798,0.002100,-0.002972,0.001075,-0.000830,-0.003603,0.000896,0.002835,0.004152,0.002494,0.000274,0.003012,0.000236,0.002472,0.007334,-0.007456,0.002126,0.001394,-0.004337,-0.007888,0.005529,0.000918,0.000720,0.000280,-0.001919,0.001508,0.001949,-0.001497,0.001230,0.000807,0.000015,-0.000064,0.001504,-0.001990,-0.000015,0.000775,-0.000523,0.002154,0.001255,0.000414,0.000577,0.000330,0.001437,0.001105,-0.000101,-0.000120,0.000657,0.000166,-0.001112,0.000889,0.000372,0.000657,0.000928,-0.000215,0.000224,0.000565,0.000330,0.000292,0.000197,-0.000598,0.000787,0.000429,0.000866,0.000944,0.000434,-0.000398,0.000217,-0.000186,-0.000214,0.001613,0.001532,0.000447,-0.000391,0.000676,-0.001097,0.000478,0.000832,0.000190,-0.000282,-0.000400,-0.000315,0.000918,-0.000315,0.000428,0.000410,0.000147,-0.000568,-0.000256,-0.000437,0.000433,-0.000034,-0.000689,0.000114,0.000350,-0.000156,0.000121,0.000029,0.000091,0.000517,-0.000020,-0.000059,-0.000363,-0.000185,0.000341,-0.000432,0.000191,-0.000258,0.000201,0.000255,0.000112,0.000332,0.000195,-0.000183,-0.000160,-0.000344,-0.000035,-0.000120,-0.000132,0.000176,0.000098,0.000209,0.000072,-0.000039,0.000108,-0.000052,-0.000006,0.000150,-0.000046,0.000253,0.000087,0.000020,9.947490e-06,0.000020,-1.545393e-07,-1.332516e-07,-3.557214e-08
1,-1.390127,-2.211286,0.641949,0.480664,-0.239196,-0.387141,2.109289,0.654540,-0.251707,-0.133599,-0.882627,0.938515,1.286549,-0.575849,0.198403,-0.318513,0.027164,-0.074123,0.297622,-0.815348,-0.735152,0.686201,0.206959,-0.296421,0.058766,-0.559377,0.551333,0.051256,0.121324,0.057215,-0.134397,0.032183,0.035923,-0.147303,-0.228000,0.889540,0.078292,-0.156148,0.020600,0.068942,0.882819,0.002429,0.141881,0.217306,0.200017,0.068899,0.026851,-0.084228,-0.157691,0.009721,0.103782,0.051113,0.018131,0.040565,0.011437,-0.032416,-0.058184,-0.036572,0.011721,-0.015764,0.003151,-0.010298,-0.026554,-0.056117,-0.010862,0.014048,-0.007604,-0.006000,0.001828,0.009058,0.044494,0.034462,0.001919,0.018852,0.012703,-0.006174,-0.010952,0.011183,0.008523,0.015606,0.007442,-0.005025,-0.000957,0.01720,0.019514,0.004926,-0.005495,0.001981,-0.000666,-0.007510,-0.004496,-0.002500,0.004563,-0.001909,-0.000678,0.006479,-0.007231,-0.007920,-0.006523,0.004197,-0.004591,-0.002859,-0

In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain_transformed, y_subtrain, validation_data=(X_val_transformed, y_val) ,batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain_transformed, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val_transformed, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 0.6173 - binary_accuracy: 0.6518 - val_loss: 0.4579 - val_binary_accuracy: 0.7836
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4465 - binary_accuracy: 0.7873 - val_loss: 0.4353 - val_binary_accuracy: 0.7898
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4306 - binary_accuracy: 0.7919 - val_loss: 0.4307 - val_binary_accuracy: 0.7920
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4275 - binary_accuracy: 0.7932 - val_loss: 0.4291 - val_binary_accuracy: 0.7927
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4259 - binary_accuracy: 0.7933 - val_loss: 0.4281 - val_binary_accuracy: 0.7935
Epoch 6/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4240 - binary_accuracy: 0.7961 - val_loss: 0.4276 - val_binary_accuracy: 0.7933
Epoch 7/100
676/676 [==============================] - 1s 

Result: De-correlated dataset performed on par + it makes the features uninterpretable (i.e. not use PCA)

##### Perform Final Dataset Transformation:

In [ ]:
# Strategy: PowerTransform + Standardize Numerical Features; Leave Binary Features as they are
preprocessing = ColumnTransformer(
    [
      ("numeric", PowerTransformer(method="yeo-johnson", standardize=True), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_subtrain)
X_subtrain = preprocessing.transform(X_subtrain)
X_val = preprocessing.transform(X_val)

preprocessing2 = ColumnTransformer(
    [
      ("numeric", PowerTransformer(method="yeo-johnson", standardize=True), numeric_features)
    ],
    remainder="passthrough", verbose=True, n_jobs=-1
).fit(X_train)
X_train = preprocessing2.transform(X_train)
X_test = preprocessing2.transform(X_test)

In [ ]:
# Re-arrange feature_names, since they have been changed by the ColumnTransformer
feature_names = numeric_feature_names + binary_feature_names

In [ ]:
# Validation
tmp = pd.DataFrame(X_test, columns=feature_names)
print(tmp.shape)
tmp.head(3)

(37033, 233)


,goal,number_of_collaborators,funding_period,days_between_created_and_launched,number_of_images,number_of_videos,number_of_audios,number_of_interactives,number_of_words,number_of_links,number_of_creator_backings,number_of_creator_projects,number_of_rewards,number_of_words_per_reward,lowest_pledge_level,highest_pledge_level,avg_months_until_reward,staff_pick,campaign_has_demo_video,campaign_has_environmental_commitments,creator_verified_identity,creator_fb_auth,creator_has_image,creator_allows_follows,facebook_linked,twitter_linked,instagram_linked,linkedin_linked,has_limited_rewards,has_shipped_rewards,has_restricted_shipping_rewards,launch_quartal_1,launch_quartal_2,launch_quartal_3,launch_quartal_4,location_Africa,location_Australia,location_Belgium,location_Canada,location_China,location_Denmark,location_France,location_Germany,location_Hong Kong,location_Ireland,location_Italy,location_Japan,location_Latin and South America,location_Mexico,location_Netherlands,location_New Zealand,location_No Location,location_Norway,location_Oceania and Antarctica,location_Rest of Asia,location_Rest of Europe,location_Singapore,location_Spain,location_Sweden,location_Switzerland,location_United Kingdom,location_United States,category_Art_Ceramics,category_Art_Conceptual Art,category_Art_Digital Art,category_Art_Illustration,category_Art_Installations,category_Art_Mixed Media,category_Art_No Subcategory,category_Art_Painting,category_Art_Performance Art,category_Art_Public Art,category_Art_Sculpture,category_Art_Social Practice,category_Art_Textiles,category_Art_Video Art,category_Comics_Anthologies,category_Comics_Comic Books,category_Comics_Events,category_Comics_Graphic Novels,category_Comics_No Subcategory,category_Comics_Webcomics,category_Crafts_Candles,category_Crafts_Crochet,category_Crafts_DIY,category_Crafts_Embroidery,category_Crafts_Glass,category_Crafts_Knitting,category_Crafts_No Subcategory,category_Crafts_Pottery,category_Crafts_Printing,category_Crafts_Quilts,category_Crafts_Stationery,category_Crafts_Taxidermy,category_Crafts_Weaving,category_Crafts_Woodworking,category_Dance_No Subcategory,category_Dance_Performances,category_Dance_Residencies,category_Dance_Spaces,category_Dance_Workshops,category_Design_Architecture,category_Design_Civic Design,category_Design_Graphic Design,category_Design_Interactive Design,category_Design_No Subcategory,category_Design_Product Design,category_Design_Toys,category_Design_Typography,category_Fashion_Accessories,category_Fashion_Apparel,category_Fashion_Childrenswear,category_Fashion_Couture,category_Fashion_Footwear,category_Fashion_Jewelry,category_Fashion_No Subcategory,category_Fashion_Pet Fashion,category_Fashion_Ready-to-wear,category_Film & Video_Action,category_Film & Video_Animation,category_Film & Video_Comedy,category_Film & Video_Documentary,category_Film & Video_Drama,category_Film & Video_Experimental,category_Film & Video_Family,category_Film & Video_Fantasy,category_Film & Video_Festivals,category_Film & Video_Horror,category_Film & Video_Movie Theaters,category_Film & Video_Music Videos,category_Film & Video_Narrative Film,category_Film & Video_No Subcategory,category_Film & Video_Romance,category_Film & Video_Science Fiction,category_Film & Video_Shorts,category_Film & Video_Television,category_Film & Video_Thrillers,category_Film & Video_Webseries,category_Food_Bacon,category_Food_Community Gardens,category_Food_Cookbooks,category_Food_Drinks,category_Food_Events,category_Food_Farmer's Markets,category_Food_Farms,category_Food_Food Trucks,category_Food_No Subcategory,category_Food_Restaurants,category_Food_Small Batch,category_Food_Spaces,category_Food_Vegan,category_Games_Gaming Hardware,category_Games_Live Games,category_Games_Mobile Games,category_Games_No Subcategory,category_Games_Playing Cards,category_Games_Puzzles,category_Games_Tabletop Games,category_Games_Video Games,category_Journalism_Audio,category_Journalism_No Subcategory,category_Journalism_Photo,cat

#### d) Test Different Model Hyperparameters

##### New Baseline:

In [ ]:
model = Sequential()
model.add(Dense(1, activation="sigmoid", kernel_regularizer=L1(l1=0.0000), kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=256, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=256, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=256, verbose=0)[1]))

Epoch 1/100
676/676 [==============================] - 2s 2ms/step - loss: 0.5602 - binary_accuracy: 0.7040 - val_loss: 0.4566 - val_binary_accuracy: 0.7780
Epoch 2/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4468 - binary_accuracy: 0.7823 - val_loss: 0.4371 - val_binary_accuracy: 0.7894
Epoch 3/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4322 - binary_accuracy: 0.7905 - val_loss: 0.4316 - val_binary_accuracy: 0.7916
Epoch 4/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4284 - binary_accuracy: 0.7924 - val_loss: 0.4297 - val_binary_accuracy: 0.7918
Epoch 5/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4265 - binary_accuracy: 0.7934 - val_loss: 0.4284 - val_binary_accuracy: 0.7935
Epoch 6/100
676/676 [==============================] - 1s 2ms/step - loss: 0.4244 - binary_accuracy: 0.7954 - val_loss: 0.4279 - val_binary_accuracy: 0.7934
Epoch 7/100
676/676 [==============================] - 1s 

Fixed Hyperparameters:
- Dataset Transformation = PowerTransformed Numerical Features; No Changes at Binary Features
- Loss Function = Binary Crossentropy (Recommended for Binary Classification)
- Learning Rate / Learning Rate Annealing = not necessary to tune, since I am using only adaptive learning rate optimizers
- Activation = Sigmoid must be used to obtain a probability score in [0,1]
- Number of Epochs = can be tuned inexpensively by using Early Stopping

Tunable Hyperparameters:
- Optimizers = Adadelta, RMSprop, RMSprop_centered, Adam, Adam_amsgrad, Adamax, Nadam
- Mini-Batch Size = typical values range between 1 and a few hundred; can be tuned independently from other hyperparameters and then be held fiex, since it mainly affects the convergence time and not the predictive performance
- Regularizer = L1, L2 (L2 penalizes larger weights more strongly, whereas L1 penalizes lower weights more strongly; L1 performs automatic feature selection)
- Regularization Rate (start with default and increase/decrease by factor 10)
- Weight Initializer = typically, a truncated initializer, such as Glorot, He, or LeCun, is used


##### Test Different Mini-Batch Sizes:

In [ ]:
# Define the logistic regression (i.e. NN without hidden layer and 1 sigmoid output neuron)
def logistic_regression(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  optimizer = params["optimizer"]
  regularizer = params["regularizer"]
  reg_rate = params["reg_rate"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if regularizer=="L1": reg = L1(reg_rate)
  if regularizer=="L2": reg = L2(reg_rate)
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=reg, kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "optimizer" : ["RMSprop"],
    "regularizer" : ["L1"],
    "reg_rate" : [0.00],
    "weight_initializer" : ["glorot_normal"],
    "batch_size" : [16, 32, 64, 128, 256, 512, 1024],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=logistic_regression, params=params, experiment_name="test1", seed=seed_value, print_params=True)

In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data[["duration", "loss", "binary_accuracy", "val_loss", "val_binary_accuracy", "optimizer", "regularizer", "reg_rate", "weight_initializer", "batch_size"]]\
.sort_values(by="val_binary_accuracy", ascending=False).round(4)

,duration,loss,binary_accuracy,val_loss,val_binary_accuracy,optimizer,regularizer,reg_rate,weight_initializer,batch_size
5,15.7162,0.4228,0.7957,0.4270,0.7940,RMSprop,L1,0.0,glorot_normal,512
2,31.9643,0.4247,0.7964,0.4287,0.7939,RMSprop,L1,0.0,glorot_normal,64
3,41.8425,0.4239,0.7964,0.4283,0.7938,RMSprop,L1,0.0,glorot_normal,128
4,21.8410,0.4229,0.7959,0.4273,0.7933,RMSprop,L1,0.0,glorot_normal,256
6,10.8992,0.4233,0.7951,0.4272,0.7933,RMSprop,L1,0.0,glorot_normal,1024
1,104.9411,0.4347,0.7942,0.4392,0.7926,RMSprop,L1,0.0,glorot_normal,32
0,110.8820,0.4433,0.7920,0.4492,0.7909,RMSprop,L1,0.0,glorot_normal,16


Result: 
- too small batch size (i.e. 16, 32) led to bad results + model building took too long
- too large batch size (i.e. 1024) led to mediocre results, but was very fast to train
- i.e. keep batch size of 512, as it provides the best trade-off between fast training and validation accuracy

##### Tune different optimizers and initializers:

In [ ]:
# Define the logistic regression (i.e. NN without hidden layer and 1 sigmoid output neuron)
def logistic_regression(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  optimizer = params["optimizer"]
  regularizer = params["regularizer"]
  reg_rate = params["reg_rate"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if regularizer=="L1": reg = L1(reg_rate)
  if regularizer=="L2": reg = L2(reg_rate)
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=reg, kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "optimizer" : ["Adadelta", "RMSprop", "RMSprop_centered", "Adam", "Adam_amsgrad", "Adamax", "Nadam"],
    "regularizer" : ["L1"],
    "reg_rate" : [0.00],
    "weight_initializer" : ["glorot_normal", "glorot_uniform", "he_normal", "he_uniform", "lecun_normal", "lecun_uniform"],
    "batch_size" : [512],
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=logistic_regression, params=params, experiment_name="test1", seed=seed_value, print_params=True)

  0%|          | 0/42 [00:00<?, ?it/s]

{'batch_size': 512, 'optimizer': 'Adadelta', 'reg_rate': 0.0, 'regularizer': 'L1', 'weight_initializer': 'glorot_normal'}
Epoch 1/100
338/338 [==============================] - 1s 3ms/step - loss: 0.6984 - binary_accuracy: 0.5178 - val_loss: 0.6953 - val_binary_accuracy: 0.5274
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6965 - binary_accuracy: 0.5205 - val_loss: 0.6937 - val_binary_accuracy: 0.5312
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6953 - binary_accuracy: 0.5262 - val_loss: 0.6919 - val_binary_accuracy: 0.5354
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6935 - binary_accuracy: 0.5318 - val_loss: 0.6900 - val_binary_accuracy: 0.5400
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6906 - binary_accuracy: 0.5366 - val_loss: 0.6880 - val_binary_accuracy: 0.5453
Epoch 6/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6894 - binary_ac

  2%|▏         | 1/42 [01:18<53:21, 78.09s/it]

{'batch_size': 512, 'optimizer': 'Adadelta', 'reg_rate': 0.0, 'regularizer': 'L1', 'weight_initializer': 'glorot_uniform'}
Epoch 1/100
338/338 [==============================] - 1s 3ms/step - loss: 0.6818 - binary_accuracy: 0.5836 - val_loss: 0.6818 - val_binary_accuracy: 0.5862
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6805 - binary_accuracy: 0.5874 - val_loss: 0.6807 - val_binary_accuracy: 0.5885
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6802 - binary_accuracy: 0.5876 - val_loss: 0.6795 - val_binary_accuracy: 0.5911
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6795 - binary_accuracy: 0.5885 - val_loss: 0.6783 - val_binary_accuracy: 0.5932
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6787 - binary_accuracy: 0.5902 - val_loss: 0.6770 - val_binary_accuracy: 0.5960
Epoch 6/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6777 - binary_a

  5%|▍         | 2/42 [02:36<52:04, 78.10s/it]

...

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
{'batch_size': 512, 'optimizer': 'Nadam', 'reg_rate': 0.0, 'regularizer': 'L1', 'weight_initializer': 'lecun_normal'}
Epoch 1/100
338/338 [==============================] - 2s 3ms/step - loss: 0.6036 - binary_accuracy: 0.6736 - val_loss: 0.5011 - val_binary_accuracy: 0.7524
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4876 - binary_accuracy: 0.7634 - val_loss: 0.4641 - val_binary_accuracy: 0.7780
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4573 - binary_accuracy: 0.7808 - val_loss: 0.4472 - val_binary_accuracy: 0.7853
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4430 - binary_accuracy: 0.7867 - val_loss: 0.4388 - val_binary_accuracy: 0.7891
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4353 - binary_accuracy: 0.7892 - val_loss: 0.4341 - val_binary_accuracy: 0.7912
Epoch 6/100
3

 98%|█████████▊| 41/42 [17:34<00:12, 12.72s/it]

{'batch_size': 512, 'optimizer': 'Nadam', 'reg_rate': 0.0, 'regularizer': 'L1', 'weight_initializer': 'lecun_uniform'}
Epoch 1/100
338/338 [==============================] - 2s 3ms/step - loss: 0.6060 - binary_accuracy: 0.6625 - val_loss: 0.5062 - val_binary_accuracy: 0.7499
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4917 - binary_accuracy: 0.7592 - val_loss: 0.4674 - val_binary_accuracy: 0.7759
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4603 - binary_accuracy: 0.7794 - val_loss: 0.4493 - val_binary_accuracy: 0.7834
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4449 - binary_accuracy: 0.7856 - val_loss: 0.4400 - val_binary_accuracy: 0.7875
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4366 - binary_accuracy: 0.7890 - val_loss: 0.4348 - val_binary_accuracy: 0.7904
Epoch 6/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4304 - binary_accur

100%|██████████| 42/42 [17:46<00:00, 25.39s/it]

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data[["duration", "loss", "binary_accuracy", "val_loss", "val_binary_accuracy", "optimizer", "regularizer", "reg_rate", "weight_initializer", "batch_size", "round_epochs"]] \
.sort_values(by="val_binary_accuracy", ascending=False).round(4)

,duration,loss,binary_accuracy,val_loss,val_binary_accuracy,optimizer,regularizer,reg_rate,weight_initializer,batch_size,round_epochs
12,24.5887,0.4226,0.7962,0.4268,0.7943,RMSprop_centered,L1,0.0,glorot_normal,512,28
14,21.6811,0.4226,0.7962,0.4268,0.7943,RMSprop_centered,L1,0.0,he_normal,512,28
16,21.1206,0.4226,0.7962,0.4268,0.7943,RMSprop_centered,L1,0.0,lecun_normal,512,28
27,21.9499,0.4227,0.7960,0.4268,0.7941,Adam_amsgrad,L1,0.0,he_uniform,512,28
25,22.0006,0.4227,0.7960,0.4268,0.7941,Adam_amsgrad,L1,0.0,glorot_uniform,512,28
29,21.9391,0.4227,0.7960,0.4268,0.7941,Adam_amsgrad,L1,0.0,lecun_uniform,512,28
6,16.1833,0.4228,0.7957,0.4270,0.7940,RMSprop,L1,0.0,glorot_normal,512,20
10,16.5251,0.4228,0.7957,0.4270,0.7940,RMSprop,L1,0.0,lecun_normal,512,20
8,16.1508,0.4228,0.7957,0.4270,0.7940,RMSprop,L1,0.0,he_normal,512,20
13,11.1188,0.4233,0.7955,0.4271,0.7935,RMSprop_centered,L1,0.0,glorot_uniform,512,13


Results:
- Adadelta perforemd notably inferior to the other optimizers -> i.e. drop
- also, Adamax and Nadam performed a little bit worse than the other optimizers in this setting -> i.e. drop
- RMSprop and Adam (plus their modified versions performed the best)
- no big difference between the truncated initializers (probably neglectible as long as the distribution is truncated
- in general: RMSprop works better with Normal and Adam better with Uniform distribution

##### Test if adding regularization can help to improve performance:

In [ ]:
# Define the logistic regression (i.e. NN without hidden layer and 1 sigmoid output neuron)
def logistic_regression(X_subtrain, y_subtrain, X_val, y_val, params):
  
  # Get hyperparameters from parameter_grid
  optimizer = params["optimizer"]
  regularizer = params["regularizer"]
  reg_rate = params["reg_rate"]
  weight_initializer = params["weight_initializer"]
  batch_size = params["batch_size"]
  if regularizer=="L1": reg = L1(reg_rate)
  if regularizer=="L2": reg = L2(reg_rate)
  if weight_initializer=="glorot_normal" : init = GlorotNormal(seed=seed_value)
  if weight_initializer=="glorot_uniform" : init = GlorotUniform(seed=seed_value)
  if weight_initializer=="he_normal" : init = HeNormal(seed=seed_value)
  if weight_initializer=="he_uniform" : init = HeUniform(seed=seed_value)
  if weight_initializer=="lecun_normal" : init = LecunNormal(seed=seed_value)
  if weight_initializer=="lecun_uniform" : init = LecunUniform(seed=seed_value)
  if optimizer=="Adadelta": opt = Adadelta()
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()

  # Define Model
  model = Sequential()
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=reg, kernel_initializer=init))

  # Compile Model
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  
  # Fit Model to Training Data
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=True, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=1, mode='max', restore_best_weights=True)])
  return history, model

In [ ]:
# Run Hyperparameter Search
params = {
    "optimizer" : ["RMSprop_centered", "Adam_amsgrad"],
    "regularizer" : ["L1", "L2"],
    "reg_rate" : [0.0001, 0.001, 0.01, 0.1, 0, 1, 10, 100],
    "weight_initializer" : ["glorot_normal", "glorot_uniform"],
    "batch_size" : [512]
}
search_object = talos.Scan(x=X_subtrain, y=y_subtrain, x_val=X_val, y_val=y_val, model=logistic_regression, params=params, experiment_name="test1", seed=seed_value, print_params=True)


  0%|          | 0/64 [00:00<?, ?it/s]

{'batch_size': 512, 'optimizer': 'RMSprop_centered', 'reg_rate': 0.0001, 'regularizer': 'L1', 'weight_initializer': 'glorot_normal'}
Epoch 1/100
338/338 [==============================] - 2s 3ms/step - loss: 0.5532 - binary_accuracy: 0.7086 - val_loss: 0.4544 - val_binary_accuracy: 0.7801
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4478 - binary_accuracy: 0.7821 - val_loss: 0.4423 - val_binary_accuracy: 0.7872
Epoch 3/100
338/338 [==============================] - 1s 3ms/step - loss: 0.4384 - binary_accuracy: 0.7889 - val_loss: 0.4383 - val_binary_accuracy: 0.7900
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4354 - binary_accuracy: 0.7901 - val_loss: 0.4367 - val_binary_accuracy: 0.7899
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4339 - binary_accuracy: 0.7915 - val_loss: 0.4356 - val_binary_accuracy: 0.7915
Epoch 6/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4319 


  2%|▏         | 1/64 [00:18<19:26, 18.52s/it]

Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
{'batch_size': 512, 'optimizer': 'RMSprop_centered', 'reg_rate': 0.0001, 'regularizer': 'L1', 'weight_initializer': 'glorot_uniform'}
Epoch 1/100
338/338 [==============================] - 2s 3ms/step - loss: 0.5554 - binary_accuracy: 0.7034 - val_loss: 0.4536 - val_binary_accuracy: 0.7802
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4469 - binary_accuracy: 0.7822 - val_loss: 0.4418 - val_binary_accuracy: 0.7868
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4380 - binary_accuracy: 0.7888 - val_loss: 0.4380 - val_binary_accuracy: 0.7904
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4352 - binary_accuracy: 0.7903 - val_loss: 0.4365 - val_binary_accuracy: 0.7904
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.4338 - binary_accuracy: 0.7913 - val_loss: 0.4355 - val_binary_accuracy: 0.79


  3%|▎         | 2/64 [00:43<21:00, 20.34s/it]

...


 97%|█████████▋| 62/64 [10:44<00:12,  6.08s/it]

{'batch_size': 512, 'optimizer': 'Adam_amsgrad', 'reg_rate': 100, 'regularizer': 'L2', 'weight_initializer': 'glorot_normal'}
Epoch 1/100
338/338 [==============================] - 1s 3ms/step - loss: 64.8165 - binary_accuracy: 0.5909 - val_loss: 0.7836 - val_binary_accuracy: 0.6501
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6963 - binary_accuracy: 0.6533 - val_loss: 0.6499 - val_binary_accuracy: 0.6501
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6485 - binary_accuracy: 0.6513 - val_loss: 0.6475 - val_binary_accuracy: 0.6501
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6473 - binary_accuracy: 0.6503 - val_loss: 0.6472 - val_binary_accuracy: 0.6501
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6474 - binary_accuracy: 0.6496 - val_loss: 0.6471 - val_binary_accuracy: 0.6501
Epoch 6/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6476 - bina


 98%|█████████▊| 63/64 [10:49<00:05,  5.98s/it]

{'batch_size': 512, 'optimizer': 'Adam_amsgrad', 'reg_rate': 100, 'regularizer': 'L2', 'weight_initializer': 'glorot_uniform'}
Epoch 1/100
338/338 [==============================] - 1s 3ms/step - loss: 69.3714 - binary_accuracy: 0.6149 - val_loss: 0.6692 - val_binary_accuracy: 0.6501
Epoch 2/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6558 - binary_accuracy: 0.6533 - val_loss: 0.6486 - val_binary_accuracy: 0.6501
Epoch 3/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6475 - binary_accuracy: 0.6513 - val_loss: 0.6472 - val_binary_accuracy: 0.6501
Epoch 4/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6471 - binary_accuracy: 0.6503 - val_loss: 0.6471 - val_binary_accuracy: 0.6501
Epoch 5/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6474 - binary_accuracy: 0.6496 - val_loss: 0.6471 - val_binary_accuracy: 0.6501
Epoch 6/100
338/338 [==============================] - 1s 2ms/step - loss: 0.6476 - bin


100%|██████████| 64/64 [10:55<00:00, 10.24s/it]


In [ ]:
# Get search results for all evaluated hyperparameter combinations sorted by val_accuracy
analyze_object = talos.Analyze(search_object)
analyze_object.data[["duration", "loss", "binary_accuracy", "val_loss", "val_binary_accuracy", "optimizer", "regularizer", "reg_rate", "weight_initializer", "batch_size", "round_epochs"]]\
.sort_values(by="val_binary_accuracy", ascending=False).round(4).head(20)

,duration,loss,binary_accuracy,val_loss,val_binary_accuracy,optimizer,regularizer,reg_rate,weight_initializer,batch_size,round_epochs
16,23.4158,0.4226,0.7962,0.4268,0.7943,RMSprop_centered,L1,0.0000,glorot_normal,512,28
18,23.3299,0.4226,0.7962,0.4268,0.7943,RMSprop_centered,L2,0.0000,glorot_normal,512,28
51,23.1804,0.4227,0.7960,0.4268,0.7941,Adam_amsgrad,L2,0.0000,glorot_uniform,512,28
49,22.8586,0.4227,0.7960,0.4268,0.7941,Adam_amsgrad,L1,0.0000,glorot_uniform,512,28
19,11.0020,0.4233,0.7955,0.4271,0.7935,RMSprop_centered,L2,0.0000,glorot_uniform,512,13
17,11.3855,0.4233,0.7955,0.4271,0.7935,RMSprop_centered,L1,0.0000,glorot_uniform,512,13
2,16.3342,0.4287,0.7944,0.4324,0.7935,RMSprop_centered,L2,0.0001,glorot_normal,512,20
3,16.2596,0.4287,0.7943,0.4324,0.7935,RMSprop_centered,L2,0.0001,glorot_uniform,512,20
1,24.3727,0.4304,0.7939,0.4342,0.7932,RMSprop_centered,L1,0.0001,glorot_uniform,512,27
50,12.9613,0.4236,0.7949,0.4274,0.7932,Adam_amsgrad,L2,0.0000,glorot_normal,512,14


Result:
- Logistic Regression with only structured features is shallow enough so that no overfitting happens
- thus, no regularization is needed
- L1 and L2 performed more or less the same (so doesn't matter which one is chosen)

#### e) Best-Found LR Structured

- Preprocessing = Yeo-Johnson Transformation + Standardization for Numeric Features; Leave Binary Features as they are
- No regularization
- GlorotNormal initialization
- RMSProp (centered) optimizer
- Batch Size: 512